# Sagemaker and Seldon Core Scikit-learn Example

 * Train a model on AWS Sagemaker
 * Deploy locally on Seldon Core
 
 This tutorial assumes you are already familiar with Sagemaker and have an AWS account you can use.
 
 Dependencies
 
  * AWS CLI
  * Docker
  * Git
  * Minikube

## Train Scikit-learn Iris Model

We will use the Sagemaker example notebook [Iris Training and Prediction with Sagemaker Scikit-learn](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb)

  1. Run this notebook inside Sagemaker but change the training step to include an ```output_path``` parameter with value an S3 bucket you have created. For example ```output_path="s3://<my-bucket>/<my-path>``` as shown below:
     * <img src="sagemaker-scikit-iris-train.png"/>
  1. Once trained you should find the model saved to the location you specified. You will need to use this S3 location later when deploying locally.

## Create Sagemaker SKLearn Image

To run your model locally you will need to create the sagemaker sklearn image

In [2]:
!git clone https://github.com/aws/sagemaker-scikit-learn-container.git

Cloning into 'sagemaker-scikit-learn-container'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
Checking connectivity... done.


Build the base container.

In [3]:
!cd sagemaker-scikit-learn-container && \
    docker build -t sklearn-base:0.20.0-cpu-py3 -f docker/0.20.0/base/Dockerfile.cpu --build-arg py_version=3 .

Sending build context to Docker daemon  238.1kB
Step 1/7 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu

7c7e4631: Pulling fs layer 
76a881a4: Pulling fs layer 
7358fbfc: Pulling fs layer 
Digest: sha256:4d8065a49c2b4c5bb225b4dd5fba3a77c119d925cfe34d5a0095388f128922d0
Status: Downloaded newer image for ubuntu:16.04
 ---> 9361ce633ff1
Step 2/7 : ARG py_version
 ---> Running in 39d0efb5f605
Removing intermediate container 39d0efb5f605
 ---> 74758aba076b
Step 3/7 : RUN test $py_version || exit 1
 ---> Running in 4a5bcd9a904d
Removing intermediate container 4a5bcd9a904d
 ---> 6e27c0b9b5fb
Step 4/7 : RUN apt-get update &&     apt-get -y install build-essential libatlas-dev git wget curl nginx jq &&     if [ $py_version -eq 2 ];        then apt-get -y install python-dev python-setuptools                      python-numpy python-scipy libatlas3-base;        else apt-get -y install python3-dev python3-setuptools                      python3-numpy python3-scipy libatlas3-base; fi
 ---> 

Get:10 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu xenial/main amd64 sgml-base all 1.26+nmu4ubuntu1 [12.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libjpeg-turbo8 amd64 1.4.2-0ubuntu3.1 [111 kB]
Get:13 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 perl-modules-5.22 all 5.22.1-9ubuntu0.6 [2629 kB]
Get:14 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libperl5.22 amd64 5.22.1-9ubuntu0.6 [3405 kB]
Get:15 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 perl amd64 5.22.1-9ubuntu0.6 [237 kB]
Get:16 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjbig0 amd64 2.1-3.1 [26.6 kB]
Get:17 http://archive.ubuntu.com/ubuntu xenial/main amd64 libgmp10 amd64 2:6.1.0+dfsg-2 [240 kB]
Get:18 http://archive.ubuntu.com/ubuntu xenial/main amd64 libmpfr4 amd64 3.1.4-1 [191 kB]
Get:19 http://archive.ubuntu.com/ubuntu xenial/main amd64 libmpc3 amd64 1.0.3-1 [39.7

Get:84 http://archive.ubuntu.com/ubuntu xenial/main amd64 cpp amd64 4:5.3.1-1ubuntu1 [27.7 kB]
Get:85 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libcc1-0 amd64 5.4.0-6ubuntu1~16.04.11 [38.8 kB]
Get:86 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgomp1 amd64 5.4.0-6ubuntu1~16.04.11 [55.0 kB]
Get:87 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libitm1 amd64 5.4.0-6ubuntu1~16.04.11 [27.4 kB]
Get:88 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libatomic1 amd64 5.4.0-6ubuntu1~16.04.11 [8896 B]
Get:89 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libasan2 amd64 5.4.0-6ubuntu1~16.04.11 [264 kB]
Get:90 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 liblsan0 amd64 5.4.0-6ubuntu1~16.04.11 [105 kB]
Get:91 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libtsan0 amd64 5.4.0-6ubuntu1~16.04.11 [244 kB]
Get:92 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libubsan0 amd64 5.4.0-6ubuntu1~16.04.

Selecting previously unselected package libjpeg-turbo8:amd64.
Preparing to unpack .../libjpeg-turbo8_1.4.2-0ubuntu3.1_amd64.deb ...
Unpacking libjpeg-turbo8:amd64 (1.4.2-0ubuntu3.1) ...
Selecting previously unselected package perl-modules-5.22.
Preparing to unpack .../perl-modules-5.22_5.22.1-9ubuntu0.6_all.deb ...
Unpacking perl-modules-5.22 (5.22.1-9ubuntu0.6) ...
Selecting previously unselected package libperl5.22:amd64.
Preparing to unpack .../libperl5.22_5.22.1-9ubuntu0.6_amd64.deb ...
Unpacking libperl5.22:amd64 (5.22.1-9ubuntu0.6) ...
Selecting previously unselected package perl.
Preparing to unpack .../perl_5.22.1-9ubuntu0.6_amd64.deb ...
Unpacking perl (5.22.1-9ubuntu0.6) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../libjbig0_2.1-3.1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1) ...
Selecting previously unselected package libgmp10:amd64.
Preparing to unpack .../libgmp10_2%3a6.1.0+dfsg-2_amd64.deb ...
Unpacking libgmp10:amd64 (2:6.1.0+df

Selecting previously unselected package libkrb5-26-heimdal:amd64.
Preparing to unpack .../libkrb5-26-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libkrb5-26-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libheimntlm0-heimdal:amd64.
Preparing to unpack .../libheimntlm0-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libheimntlm0-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libgssapi3-heimdal:amd64.
Preparing to unpack .../libgssapi3-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libgssapi3-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libsasl2-modules-db:amd64.
Preparing to unpack .../libsasl2-modules-db_2.1.26.dfsg1-14ubuntu0.1_amd64.deb ...
Unpacking libsasl2-modules-db:amd64 (2.1.26.dfsg1-14ubuntu0.1) ...
Selecting previously unselected package libsasl2-2:amd64.

Selecting previously unselected package patch.
Preparing to unpack .../patch_2.7.5-1ubuntu0.16.04.1_amd64.deb ...
Unpacking patch (2.7.5-1ubuntu0.16.04.1) ...
Selecting previously unselected package dpkg-dev.
Preparing to unpack .../dpkg-dev_1.18.4ubuntu1.5_all.deb ...
Unpacking dpkg-dev (1.18.4ubuntu1.5) ...
Selecting previously unselected package build-essential.
Preparing to unpack .../build-essential_12.1ubuntu2_amd64.deb ...
Unpacking build-essential (12.1ubuntu2) ...
Selecting previously unselected package curl.
Preparing to unpack .../curl_7.47.0-1ubuntu2.12_amd64.deb ...
Unpacking curl (7.47.0-1ubuntu2.12) ...
Selecting previously unselected package libfakeroot:amd64.
Preparing to unpack .../libfakeroot_1.20.2-1ubuntu1_amd64.deb ...
Unpacking libfakeroot:amd64 (1.20.2-1ubuntu1) ...
Selecting previously unselected package fakeroot.
Preparing to unpack .../fakeroot_1.20.2-1ubuntu1_amd64.deb ...
Unpacking fakeroot (1.20.2-1ubuntu1) ...
Selecting previously unselected package fonts

Setting up libhogweed4:amd64 (3.2-1ubuntu0.16.04.1) ...
Setting up libidn11:amd64 (1.32-3ubuntu1.2) ...
Setting up libp11-kit0:amd64 (0.23.2-5~ubuntu16.04.1) ...
Setting up libtasn1-6:amd64 (4.7-3ubuntu0.16.04.3) ...
Setting up libgnutls30:amd64 (3.4.10-4ubuntu1.4) ...
Setting up libpng12-0:amd64 (1.2.54-1ubuntu1.1) ...
Setting up libsqlite3-0:amd64 (3.11.0-1ubuntu1.1) ...
Setting up libssl1.0.0:amd64 (1.0.2g-1ubuntu4.15) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up libxtables11:amd64 (1.6.0-2ubuntu3) ...
Setting up netbase (5.3) ...
Setting up ucf (3.0036) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up openssl (1.0.2g-1ubuntu4.15) ...
Setting up ca-certificates (20170717~16.04.2) ...
debconf: unable to initialize frontend: Dialog
debconf

0 upgraded, 24 newly installed, 0 to remove and 4 not upgraded.
Need to get 57.1 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython3.5-minimal amd64 3.5.2-2ubuntu0~16.04.5 [524 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3.5-minimal amd64 3.5.2-2ubuntu0~16.04.5 [1598 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-minimal amd64 3.5.1-3 [23.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial/main amd64 mime-support all 3.59ubuntu1 [31.0 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial/main amd64 libmpdec2 amd64 2.4.2-1 [82.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython3.5-stdlib amd64 3.5.2-2ubuntu0~16.04.5 [2134 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3.5 amd64 3.5.2-2ubuntu0~16.04.5 [165 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpython3-

running python post-rtupdate hooks for python3.5...
Setting up python3-decorator (4.0.6-1) ...
Setting up python3-dev (3.5.1-3) ...
Setting up python3-numpy (1:1.11.0-1ubuntu1) ...
Setting up python3-pkg-resources (20.7.0-1) ...
Setting up python3-setuptools (20.7.0-1) ...
Setting up python3-scipy (0.17.0-1) ...
Setting up dh-python (2.20151103ubuntu1.1) ...
Processing triggers for libc-bin (2.23-0ubuntu11) ...
Removing intermediate container 5aea2afd75bb
 ---> cac7438c0b56
Step 5/7 : RUN cd /tmp &&      curl -O https://bootstrap.pypa.io/get-pip.py &&      if [ $py_version -eq 2 ];         then python2 get-pip.py;         else python3 get-pip.py; fi &&      rm get-pip.py
 ---> Running in ae1f8396d583
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1659k  100 1659k    0     0  1489k      0  0:00:01  0:00:01 --:--:-- 1489k    0
Removing intermediate container ae1f8396d583
 --

In [4]:
!cd sagemaker-scikit-learn-container && python setup.py bdist_wheel

running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/serving.py -> build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/__init__.py -> build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/training.py -> build/lib/sagemaker_sklearn_container
installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/wheel
creating build/bdist.linux-x86_64/wheel/sagemaker_sklearn_container
copying build/lib/sagemaker_sklearn_container/serving.py -> build/bdist.linux-x86_64/wheel/sagemaker_sklearn_container
copying build/lib/sagemaker_sklearn_container/__init__.py -> build/bdist.linux-x86_64/wheel/sagemaker_sklearn_container
copying build/lib/sagemaker_sklearn_container/training.py -> build/bdist.linux-x86_64/wheel/sagemaker_sklearn_container
running 

In [5]:
!cd sagemaker-scikit-learn-container && docker build -t sklearn-final:0.20.0-cpu-py3 -f docker/0.20.0/final/Dockerfile.cpu --build-arg py_version=3 .

Sending build context to Docker daemon  275.5kB
Step 1/7 : ARG py_version
Step 2/7 : FROM sklearn-base:0.20.0-cpu-py$py_version
 ---> ceba4ba850cf
Step 3/7 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Running in bfc6184c7edd
Removing intermediate container bfc6184c7edd
 ---> da04d67b8df6
Step 4/7 : COPY dist/sagemaker_sklearn_container-1.0-py2.py3-none-any.whl /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl
 ---> 764fa3089bf1
Step 5/7 : RUN pip install --no-cache /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl &&     rm /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl
 ---> Running in a4548c037c0d
Processing /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl


  Running setup.py install for inotify-simple: started
    Running setup.py install for inotify-simple: finished with status 'done'
  Running setup.py install for pycparser: started
    Running setup.py install for pycparser: finished with status 'done'
  Running setup.py install for psutil: started
    Running setup.py install for psutil: finished with status 'done'
  Running setup.py install for sagemaker-containers: started
    Running setup.py install for sagemaker-containers: finished with status 'done'
Removing intermediate container a4548c037c0d
 ---> 1b0693ec2db7
Step 6/7 : ENV SAGEMAKER_TRAINING_MODULE sagemaker_sklearn_container.training:main
 ---> Running in 9bf0253661c4
Removing intermediate container 9bf0253661c4
 ---> 216ce5477449
Step 7/7 : ENV SAGEMAKER_SERVING_MODULE sagemaker_sklearn_container.serving:main
 ---> Running in e5d2b42fbce6
Removing intermediate container e5d2b42fbce6
 ---> 7e31b3d9f5f5
Successfully built 7e31b3d9f5f5
Successfully tagged sklearn-final:0.20

**Specify the location of the model.tar.gz that AWS Sagemaker created on S3** An example is shown below.

In [1]:
%env SAGEMAKER_MODEL_DIRECTORY=s3://seldon-sagemaker-testing/scikit_learn_iris/sagemaker-scikit-learn-2019-01-04-19-26-40-470/output/model.tar.gz

env: SAGEMAKER_MODEL_DIRECTORY=s3://seldon-sagemaker-testing/scikit_learn_iris/sagemaker-scikit-learn-2019-01-04-19-26-40-470/output/model.tar.gz


## Prepare Prediction Code

We will create the methods required to load and predict against our trained model as a module that can be loaded by the Sagemaker scikit-learn-container image we created.

In [2]:
!pygmentize scikit_learn_iris/scikit_learn_iris.py

from sklearn.externals import joblib
import numpy as np
import os
import logging
from sagemaker_containers.beta.framework import files

logging.basicConfig(format='%(asctime)s %(levelname)s - %(name)s - %(message)s', level=logging.INFO)

logging.getLogger('boto3').setLevel(logging.INFO)
logging.getLogger('s3transfer').setLevel(logging.INFO)
logging.getLogger('botocore').setLevel(logging.WARN)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Example predict function that extracts a model into the provided model_dir
def model_fn(model_dir):
    model_dir_env = os.environ["SAGEMAKER_MODEL_DIRECTORY"]
    logger.info("Using model directory %s",model_dir_env)
    files.download_and_extract(model_dir_env, "", model_dir)
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def predict_fn(data, model):
    if len(data.shape) == 1:
        data = np.reshape(data,(1,data.shape[0]))
    return model.predict(data)


Package the code as a tar.gz file

In [8]:
!cd scikit_learn_iris && tar -cvf ../scikit_learn_iris_code.tar .
!gzip -f scikit_learn_iris_code.tar

./
./scikit_learn_iris.py


**Set the path to store the code**

In [3]:
%env SAGEMAKER_SUBMIT_DIRECTORY=s3://seldon-sagemaker-testing/scikit_learn_iris_code.tar.gz

env: SAGEMAKER_SUBMIT_DIRECTORY=s3://seldon-sagemaker-testing/scikit_learn_iris_code.tar.gz


In [10]:
!aws s3 cp scikit_learn_iris_code.tar.gz ${SAGEMAKER_SUBMIT_DIRECTORY}

upload: ./scikit_learn_iris_code.tar.gz to s3://seldon-sagemaker-testing/scikit_learn_iris_code.tar.gz


We can now do a local test with Docker. To make this work you will need to have your aws credentials in ~/.aws/config.

In [4]:
!docker run -d --rm -p 8080:8080 \
        -e SAGEMAKER_MODEL_DIRECTORY=${SAGEMAKER_MODEL_DIRECTORY} \
        -e SAGEMAKER_SUBMIT_DIRECTORY=${SAGEMAKER_SUBMIT_DIRECTORY} \
        -e SAGEMAKER_PROGRAM="scikit_learn_iris.py" \
        -v ~/.aws:/root/.aws \
        --name "scikit_predictor" \
        sklearn-final:0.20.0-cpu-py3 serve

8dc0e37abaa1f5ac92fc96cc287b4d33d2fed4bf9ddbb8dde4b04e4f6f5571f0


Test the running image with a prediction call

In [5]:
!curl 0.0.0.0:8080/invocations -d '1.0,2.0,3.0,4.0' -H "Content-Type: text/csv"

[1.0]

In [6]:
!docker rm -f scikit_predictor

scikit_predictor


## Test with Seldon-Core in Minikube

In [14]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Wrap Model and Test

Recreate the Sagemaker scikit-learn base and final containers inside Minikube. In a production scenario you would build these images and push to your DockerHub or private repo.

In [12]:
!eval $(minikube docker-env) && \
    cd sagemaker-scikit-learn-container && \
    docker build -t sklearn-base:0.20.0-cpu-py3 -f docker/0.20.0/base/Dockerfile.cpu --build-arg py_version=3 .

Sending build context to Docker daemon  275.5kB
Step 1/7 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu

7c7e4631: Pulling fs layer 
76a881a4: Pulling fs layer 
7358fbfc: Pulling fs layer 
Digest: sha256:58d0da8bc2f434983c6ca4713b08be00ff5586eb5cdff47bcde4b2e88fd40f88
Status: Downloaded newer image for ubuntu:16.04
 ---> 9361ce633ff1
Step 2/7 : ARG py_version
 ---> Running in 1fc359ec522a
Removing intermediate container 1fc359ec522a
 ---> 0d53a0355dc9
Step 3/7 : RUN test $py_version || exit 1
 ---> Running in d44f818ef17c
Removing intermediate container d44f818ef17c
 ---> 2092375bbf59
Step 4/7 : RUN apt-get update &&     apt-get -y install build-essential libatlas-dev git wget curl nginx jq &&     if [ $py_version -eq 2 ];        then apt-get -y install python-dev python-setuptools                      python-numpy python-scipy libatlas3-base;        else apt-get -y install python3-dev python3-setuptools                      python3-numpy python3-scipy libatlas3-base; fi
 ---> 

Get:8 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libx11-data all 2:1.6.3-1ubuntu2.1 [113 kB]
Get:9 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libx11-6 amd64 2:1.6.3-1ubuntu2.1 [570 kB]
Get:10 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu xenial/main amd64 sgml-base all 1.26+nmu4ubuntu1 [12.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libjpeg-turbo8 amd64 1.4.2-0ubuntu3.1 [111 kB]
Get:13 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 perl-modules-5.22 all 5.22.1-9ubuntu0.6 [2629 kB]
Get:14 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libperl5.22 amd64 5.22.1-9ubuntu0.6 [3405 kB]
Get:15 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 perl amd64 5.22.1-9ubuntu0.6 [237 kB]
Get:16 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjbig0 amd64 2.1-3.1 [26.6 kB]
Get:17 http://archive.ubuntu.com/ubuntu xenial/main amd

Get:82 http://archive.ubuntu.com/ubuntu xenial/main amd64 libisl15 amd64 0.16.1-1 [524 kB]
Get:83 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 cpp-5 amd64 5.4.0-6ubuntu1~16.04.11 [7660 kB]
Get:84 http://archive.ubuntu.com/ubuntu xenial/main amd64 cpp amd64 4:5.3.1-1ubuntu1 [27.7 kB]
Get:85 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libcc1-0 amd64 5.4.0-6ubuntu1~16.04.11 [38.8 kB]
Get:86 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgomp1 amd64 5.4.0-6ubuntu1~16.04.11 [55.0 kB]
Get:87 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libitm1 amd64 5.4.0-6ubuntu1~16.04.11 [27.4 kB]
Get:88 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libatomic1 amd64 5.4.0-6ubuntu1~16.04.11 [8896 B]
Get:89 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libasan2 amd64 5.4.0-6ubuntu1~16.04.11 [264 kB]
Get:90 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 liblsan0 amd64 5.4.0-6ubuntu1~16.04.11 [105 kB]
Get:91 http://

Selecting previously unselected package libxext6:amd64.
Preparing to unpack .../libxext6_2%3a1.3.3-1_amd64.deb ...
Unpacking libxext6:amd64 (2:1.3.3-1) ...
Selecting previously unselected package sgml-base.
Preparing to unpack .../sgml-base_1.26+nmu4ubuntu1_all.deb ...
Unpacking sgml-base (1.26+nmu4ubuntu1) ...
Selecting previously unselected package libjpeg-turbo8:amd64.
Preparing to unpack .../libjpeg-turbo8_1.4.2-0ubuntu3.1_amd64.deb ...
Unpacking libjpeg-turbo8:amd64 (1.4.2-0ubuntu3.1) ...
Selecting previously unselected package perl-modules-5.22.
Preparing to unpack .../perl-modules-5.22_5.22.1-9ubuntu0.6_all.deb ...
Unpacking perl-modules-5.22 (5.22.1-9ubuntu0.6) ...
Selecting previously unselected package libperl5.22:amd64.
Preparing to unpack .../libperl5.22_5.22.1-9ubuntu0.6_amd64.deb ...
Unpacking libperl5.22:amd64 (5.22.1-9ubuntu0.6) ...
Selecting previously unselected package perl.
Preparing to unpack .../perl_5.22.1-9ubuntu0.6_amd64.deb ...
Unpacking perl (5.22.1-9ubuntu0.

Selecting previously unselected package libhx509-5-heimdal:amd64.
Preparing to unpack .../libhx509-5-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libhx509-5-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libkrb5-26-heimdal:amd64.
Preparing to unpack .../libkrb5-26-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libkrb5-26-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libheimntlm0-heimdal:amd64.
Preparing to unpack .../libheimntlm0-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libheimntlm0-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libgssapi3-heimdal:amd64.
Preparing to unpack .../libgssapi3-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libgssapi3-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected pa

Selecting previously unselected package libdpkg-perl.
Preparing to unpack .../libdpkg-perl_1.18.4ubuntu1.5_all.deb ...
Unpacking libdpkg-perl (1.18.4ubuntu1.5) ...
Selecting previously unselected package xz-utils.
Preparing to unpack .../xz-utils_5.1.1alpha+20120614-2ubuntu2_amd64.deb ...
Unpacking xz-utils (5.1.1alpha+20120614-2ubuntu2) ...
Selecting previously unselected package patch.
Preparing to unpack .../patch_2.7.5-1ubuntu0.16.04.1_amd64.deb ...
Unpacking patch (2.7.5-1ubuntu0.16.04.1) ...
Selecting previously unselected package dpkg-dev.
Preparing to unpack .../dpkg-dev_1.18.4ubuntu1.5_all.deb ...
Unpacking dpkg-dev (1.18.4ubuntu1.5) ...
Selecting previously unselected package build-essential.
Preparing to unpack .../build-essential_12.1ubuntu2_amd64.deb ...
Unpacking build-essential (12.1ubuntu2) ...
Selecting previously unselected package curl.
Preparing to unpack .../curl_7.47.0-1ubuntu2.12_amd64.deb ...
Unpacking curl (7.47.0-1ubuntu2.12) ...
Selecting previously unselecte

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up libbsd0:amd64 (0.8.2-1) ...
Setting up libexpat1:amd64 (2.1.0-7ubuntu0.16.04.3) ...
Setting up libffi6:amd64 (3.2.1-4) ...
Setting up libnettle6:amd64 (3.2-1ubuntu0.16.04.1) ...
Setting up libhogweed4:amd64 (3.2-1ubuntu0.16.04.1) ...
Setting up libidn11:amd64 (1.32-3ubuntu1.2) ...
Setting up libp11-kit0:amd64 (0.23.2-5~ubuntu16.04.1) ...
Setting up libtasn1-6:amd64 (4.7-3ubuntu0.16.04.3) ...
Setting up libgnutls30:amd64 (3.4.10-4ubuntu1.4) ...
Setting up libpng12-0:amd64 (1.2.54-1ubuntu1.1) ...
Setting up libsqlite3-0:amd64 (3.11.0-1ubuntu1.1) ...
Setting up libssl1.0.0:amd64 (1.0.2g-1ubuntu4.15) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up libxtables11:amd64 (1.6.0-2ubuntu3) ...
Setting up netba

0 upgraded, 24 newly installed, 0 to remove and 6 not upgraded.
Need to get 57.1 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython3.5-minimal amd64 3.5.2-2ubuntu0~16.04.5 [524 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3.5-minimal amd64 3.5.2-2ubuntu0~16.04.5 [1598 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-minimal amd64 3.5.1-3 [23.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial/main amd64 mime-support all 3.59ubuntu1 [31.0 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial/main amd64 libmpdec2 amd64 2.4.2-1 [82.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython3.5-stdlib amd64 3.5.2-2ubuntu0~16.04.5 [2134 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3.5 amd64 3.5.2-2ubuntu0~16.04.5 [165 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpython3-

Setting up libpython3-dev:amd64 (3.5.1-3) ...
Setting up python3.5-dev (3.5.2-2ubuntu0~16.04.5) ...
Setting up python3 (3.5.1-3) ...
running python rtupdate hooks for python3.5...
running python post-rtupdate hooks for python3.5...
Setting up python3-decorator (4.0.6-1) ...
Setting up python3-dev (3.5.1-3) ...
Setting up python3-numpy (1:1.11.0-1ubuntu1) ...
Setting up python3-pkg-resources (20.7.0-1) ...
Setting up python3-setuptools (20.7.0-1) ...
Setting up python3-scipy (0.17.0-1) ...
Setting up dh-python (2.20151103ubuntu1.1) ...
Processing triggers for libc-bin (2.23-0ubuntu11) ...
Removing intermediate container 98304ed5ca82
 ---> 048fe8f57074
Step 5/7 : RUN cd /tmp &&      curl -O https://bootstrap.pypa.io/get-pip.py &&      if [ $py_version -eq 2 ];         then python2 get-pip.py;         else python3 get-pip.py; fi &&      rm get-pip.py
 ---> Running in d88a72472ffd
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                              

In [13]:
!eval $(minikube docker-env) && \
    cd sagemaker-scikit-learn-container && \
    docker build -t sklearn-final:0.20.0-cpu-py3 -f docker/0.20.0/final/Dockerfile.cpu --build-arg py_version=3 .

Sending build context to Docker daemon  275.5kB
Step 1/7 : ARG py_version
Step 2/7 : FROM sklearn-base:0.20.0-cpu-py$py_version
 ---> 3b30a8018255
Step 3/7 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Running in d7dacaa645e8
Removing intermediate container d7dacaa645e8
 ---> add10439f4af
Step 4/7 : COPY dist/sagemaker_sklearn_container-1.0-py2.py3-none-any.whl /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl
 ---> 827ce81bf6e0
Step 5/7 : RUN pip install --no-cache /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl &&     rm /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl
 ---> Running in e9bc4fd9a4e4
Processing /sagemaker_sklearn_container-1.0-py2.py3-none-any.whl


  Stored in directory: /tmp/pip-ephem-wheel-cache-t2v0vdv8/wheels/ce/cf/f3/c0e4229f87f3b7242652850588b110c467e98c0ba2195827bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-t2v0vdv8/wheels/11/5d/7e/f325d21ffec71263368df56a2af6ab3c802e0e869abd4b3c03
  Stored in directory: /tmp/pip-ephem-wheel-cache-t2v0vdv8/wheels/d2/71/40/9c6993129f8cda369d0f21c46a13a6adab7fb1664fe6512551
  Stored in directory: /tmp/pip-ephem-wheel-cache-t2v0vdv8/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built sagemaker-containers inotify-simple psutil pycparser
Removing intermediate container e9bc4fd9a4e4
 ---> 8d85d90db70a
Step 6/7 : ENV SAGEMAKER_TRAINING_MODULE sagemaker_sklearn_container.training:main
 ---> Running in 20b90ac2d271
Removing intermediate container 20b90ac2d271
 ---> cf49fd3dc891
Step 7/7 : ENV SAGEMAKER_SERVING_MODULE sagemaker_sklearn_container.serving:main
 ---> Running in 115eb419c122
Removing intermediate container 115eb419c122
 ---> 5a9e0462ccf5
Successf

## Create AWS Credential Secret from Template


In [ ]:
!cp aws-config-secret.yaml.tmpl aws-config-secret.yaml

Edit aws-config-secret.yaml and fill in the base64 encoded values, for example you can run on linux:

 ``` echo -n <my_aws_access_id> | base64```

Take the value and place into the file.

Once completed you can create the secret via ```kubectl```

In [14]:
!kubectl create -f aws-config-secret.yaml

secret/aws-config created


## Create Seldon Deployment from Template

We will use the environment variables you set above to fill in the details for the Seldon Deployment so the Sagemake image container knows where to download your code.

In [17]:
!cat scikit_learn_iris_deployment.json.tmpl | sed s#{SAGEMAKER_MODEL_DIRECTORY}#${SAGEMAKER_MODEL_DIRECTORY}# | sed s#{SAGEMAKER_SUBMIT_DIRECTORY}#${SAGEMAKER_SUBMIT_DIRECTORY}# > scikit_learn_iris_deployment.json

In [18]:
!kubectl create -f scikit_learn_iris_deployment.json

seldondeployment.machinelearning.seldon.io/sklearn-sagemaker created


## Test Predictive Endpoint

In [19]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    sklearn-sagemaker --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[7.819 3.729 7.32  0.637]]
RECEIVED RESPONSE:
meta {
  puid: "gppntf4efgvmv2qfgka8hpfs7l"
  requestPath {
    key: "sagemaker-proxy"
    value: "seldonio/sagemaker-proxy:0.1"
  }
}
data {
  names: "t:0"
  ndarray {
    values {
      list_value {
        values {
          number_value: 2.0
        }
      }
    }
  }
}




## Teardown

In [ ]:
!minikube delete